In [1]:
import sqlite3 as sql
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 500)
pd.options.display.max_rows
conn = sql.connect('librairieImmobilier.db')

cursor = conn.cursor()

# 1. Nombre total d’appartements vendus au 1er semestre 2020

In [3]:
pd.read_sql_query("SELECT  COUNT(*) as 'Nombre ventes' FROM Mutation INNER JOIN Batiment ON Mutation.Id = Batiment.IdMutation AND \"Nature mutation\"='Vente' AND \"Code type local\" = 2 ",conn)

Nombre ventes
0         203137

# 2. Proportion des ventes d’appartements par le nombre de pièces

In [4]:
pd.read_sql_query("SELECT \"Nombre pieces principales\",CAST(COUNT(*) as REAL)  / (SELECT COUNT(*) FROM Mutation AS 'Proportion vente') AS 'Proportion ventes' FROM Mutation INNER JOIN Batiment ON IdMutation = Id GROUP BY \"Nombre pieces principales\"",conn)

Nombre pieces principales  Proportion ventes
0                         NaN       4.662932e-01
1                         0.0       1.606172e-01
2                         1.0       3.843189e-02
3                         2.0       6.614268e-02
4                         3.0       8.920165e-02
5                         4.0       9.077229e-02
6                         5.0       5.435334e-02
7                         6.0       2.123959e-02
8                         7.0       7.737113e-03
9                         8.0       3.025167e-03
10                        9.0       1.122598e-03
11                       10.0       5.131170e-04
12                       11.0       2.470868e-04
13                       12.0       1.375450e-04
14                       13.0       5.106461e-05
15                       14.0       3.459216e-05
16                       15.0       1.729608e-05
17                       16.0       1.647246e-05
18                       17.0       1.070710e-05
19                       18.0       6.588983e-06
20                       19.0       1.647246e-06
21                       20.0       6.588983e-06
22                       21.0       2.470868e-06
23                       22.0       3.294491e-06
24                       23.0       2.470868e-06
25                       24.0       2.470868e-06
26                       25.0       1.647246e-06
27                       28.0       1.647246e-06
28                       30.0       1.647246e-06
29                       33.0       8.236228e-07
30                       41.0       8.236228e-07
31                       54.0       1.647246e-06
32                       55.0       8.236228e-07
33                       70.0       8.236228e-07

# 3. Liste des 10 départements où le prix du mètre carré est le plus élevé

In [5]:
pd.read_sql_query("SELECT  AVG(\"Valeur fonciere\")/AVG(\"Surface reelle bati\") as 'Prix m/2',\"Code departement\"  FROM Mutation INNER JOIN Batiment ON Mutation.Id = Batiment.IdMutation GROUP BY \"Code departement\" ORDER BY \"Prix m/2\"   LIMIT 10",conn)

Prix m/2 Code departement
0   795.959448               43
1   807.394371               55
2   849.651880               70
3   924.100056               79
4   978.914434               52
5  1016.041209               36
6  1055.459866                7
7  1062.703224                8
8  1074.286738               19
9  1079.343513               48

# 4. Prix moyen du mètre carré d’une maison en Île-de-France

In [6]:
pd.read_sql_query("SELECT AVG(\"Valeur fonciere\")/AVG(\"Surface reelle bati\") as 'Prix m/2'FROM Mutation INNER JOIN Batiment ON Mutation.Id = Batiment.IdMutation AND \"Code departement\" IN(75,77,78,91,92,93,94,95)",conn)

Prix m/2
0  32814.510592

# 5. Liste des 10 appartements les plus chers avec le département et le nombre de mètres carrés

In [8]:
pd.read_sql_query("SELECT  \"Valeur fonciere\",\"Code departement\",\"Surface Carrez du 1er lot\"  FROM Mutation INNER JOIN Batiment ON Mutation.Id = Batiment.IdMutation  ORDER BY \"Prix m/2\"   LIMIT 10",conn)

Valeur fonciere Code departement Surface Carrez du 1er lot
0         8000,00                1                      None
1         2175,00                1                      None
2         2175,00                1                      None
3         2175,00                1                      None
4         2175,00                1                      None
5         2175,00                1                      None
6        75000,00                1                      None
7          123,00                1                      None
8          123,00                1                      None
9        72000,00                1                      None

# 6. Taux d’évolution du nombre de ventes entre le premier et le second trimestre de 2020

In [9]:
pd.read_sql_query("SELECT (SELECT CAST(COUNT(*) as REAL)FROM Mutation WHERE substr(\"Date mutation\",9,9) >= '04') / (SELECT COUNT(*) FROM Mutation WHERE substr(\"Date mutation\",9,9) < '04' ) AS 'Taux évolution'",conn)

Taux évolution
0           None

# 7. Liste des communes où le nombre de ventes a augmenté d'au moins 20% entre le premier et le second trimestre de 2020

In [10]:
pd.read_sql_query("SELECT Avant.Commune , (CAST(Avant AS REAL)/Apres) AS Augmentation FROM (SELECT DISTINCT Commune,COUNT(*) as Avant FROM Batiment INNER JOIN Mutation ON Id = IdMutation AND substr(\"Date mutation\",9,9) >= '04' GROUP BY Commune) as Avant INNER JOIN (SELECT DISTINCT Commune,COUNT(*) as Apres FROM Batiment INNER JOIN Mutation ON Id = IdMutation AND substr(\"Date mutation\",9,9) < '04' GROUP BY Commune ) as Apres ON Apres.Commune = Avant.Commune AND Augmentation > 1.2",conn)

Empty DataFrame
Columns: [Commune, Augmentation]
Index: []

# 8. Différence en pourcentage du prix au mètre carré entre un appartement de 2 pièces et un appartement de 3 pièces

In [11]:
pd.read_sql_query("SELECT (SELECT  AVG(\"Valeur fonciere\") / AVG(\"Surface Carrez du 1er lot\")  FROM Mutation INNER JOIN Batiment ON Id = IdMutation AND \"Nombre pieces principales\" = 2) * 100 / (SELECT  AVG(\"Valeur fonciere\") / AVG(\"Surface Carrez du 1er lot\")  FROM Mutation INNER JOIN Batiment ON Id = IdMutation AND \"Nombre pieces principales\" = 3)  AS 'Difference prix en %' ",conn)

Difference prix en %
0            165.622634

# 9. Les moyennes de valeurs foncières pour le top 3 des communes des départements 6, 13, 33, 59 et 69

In [12]:
pd.read_sql_query("SELECT Commune,AVG(\"Valeur fonciere\") as Moyenne FROM Mutation INNER JOIN Batiment ON Id =  IdMutation AND \"Code departement\" IN (6,13,33,59,69) GROUP BY Commune ORDER BY Moyenne DESC LIMIT 3",conn)

Commune       Moyenne
0  SAINT-ETIENNE-DE-LISSE  2.360296e+07
1                VALBONNE  6.081097e+06
2        PORT-SAINT-LOUIS  6.058935e+06